In [163]:
import os
import numpy as np
import pandas as pd
from pprint import pprint
from pyspark.sql.types import *
from graphframes import *

def open_dataset(file_name):
    return pd.read_csv(filepath_or_buffer=file_name, delimiter="\t", encoding="utf-8", header=0)

itineraries = open_dataset('data_sncf/timetables.csv')
itineraries = itineraries.sort_values(by=['trip_id', 'trajet', 'duree'], ascending=[True, True, True])

# refacto du dataframe
trajet_splited = itineraries['trajet'].str.split(' - ')

itineraries['start'] = trajet_splited.str[0]
itineraries['end'] = trajet_splited.str[1]

itineraries.drop('trajet', axis=1, inplace=True)

itineraries

,trip_id,duree,start,end
0,OCESN003100F140147152,138,Gare de Le Havre,Gare de Paris-St-Lazare
1,OCESN003190F040047309,145,Gare de Dieppe,Gare de Paris-St-Lazare
2,OCESN003198F030037315,97,Gare de Paris-St-Lazare,Gare de Rouen-Rive-Droite
3,OCESN003300F030037323,194,Gare de Cherbourg,Gare de Paris-St-Lazare
4,OCESN003313F380387526,149,Gare de Caen,Gare de Paris-St-Lazare
...,...,...,...,...
1570,OCESN895822F0500552575,244,Gare de Belfort-Ville,Gare de Lyon-Perrache
1571,OCESN895830F0200252600,103,Gare de Lons-le-Saunier,Gare de Lyon-Perrache
1572,OCESN895880F0500552634,144,Gare de Belfort-Ville,Gare de Lons-le-Saunier
1573,OCESN895940F0200252654,89,Gare de Besançon-Viotte,Gare de Lons-le-Saunier


In [164]:
findTravel = itineraries.loc[itineraries['trip_id'] == "OCESN014033F0900915597"]
print(findTravel)

                   trip_id  duree            start            end
19  OCESN014033F0900915597     72  Gare de Orléans  Gare de Tours


In [165]:
locsStart = itineraries['start'].tolist()
locsEnd = itineraries['end'].tolist()
locs = list(dict.fromkeys(locsStart + locsEnd))
# len(locs)
locs



['Gare de Le Havre',
 'Gare de Dieppe',
 'Gare de Paris-St-Lazare',
 'Gare de Cherbourg',
 'Gare de Caen',
 'Gare de Granville',
 'Gare de Dreux',
 'Gare de Bourges',
 'Gare de Paris-Austerlitz',
 'Gare de Nevers',
 'Gare de Argentan',
 'Gare de Orléans',
 'Gare de Aubrais-(les)',
 'Gare de Paris-Bercy',
 'Gare de Calais Ville',
 'Gare de Boulogne Ville',
 'Gare de Amiens',
 'Gare de Cambrai Ville',
 'Gare de Maubeuge',
 'Gare de Paris Gare du Nord',
 'Gare de Compiègne',
 'Gare de Le Mans',
 'Gare de Nogent-le-Rotrou',
 'Gare de Chartres',
 'Gare de Blois-Chambord',
 'Gare de Lyon-Perrache',
 'Gare de Latour-de-Carol-Enveitg',
 'Gare de Font-Romeu-Odeillo-Via',
 'Gare de Briançon',
 'Gare de Romans-Bourg-de-Péage',
 'Gare de Gap',
 'Gare de Aix-en-Provence',
 'Gare de Marseille-St-Charles',
 'Gare de Les Arcs-Draguignan',
 'Gare de Grenoble',
 'Gare de Chambéry-Chal.-les-Eaux',
 'Gare de Annecy',
 'Gare de Avignon-Centre',
 'Gare de Lyon-Part-Dieu',
 'Gare de Dijon-Ville',
 'Gare de M

In [209]:
def get_itinerary(start):
    return itineraries.loc[itineraries['start'] == start]


In [259]:
# update matrix from itineraries
def updateMatrixNodes(matrix, itineraries):
    print(itineraries)
    
    for index, row in itineraries.iterrows():
        # print(matrix[row['end']]['distanceFromSource'])
        oldValue = matrix[row['end']]['distanceFromSource']
        
        if oldValue == 'infinite':
            matrix[row['end']]['distanceFromSource'] = row['duree'],
        else:   
            matrix[row['end']]['distanceFromSource'] = oldValue + row['duree'],
        
    # TODO : processed False pour le plus petit durée 
    
    # return matrix

In [260]:


def dijsktra(df, source, destination):
    matrixLoc = {}
    
    locsStart = df['start'].tolist()
    locsEnd = df['end'].tolist()
    locs = list(dict.fromkeys(locsStart + locsEnd))
    
    # If multi start & end locations possible with this location name, get the first result
    locStart = list(filter(lambda k: source in k, locs))[0]
    locEnd = list(filter(lambda k: destination in k, locs))[0]
    
    # print(locStart)
    # print(locEnd)
    # print(locs)
    
    # initialize matrix
    for loc in locs:
        matrixLoc[loc] = {
            'distanceFromSource': 'infinite',
            'fromLoc': '',
            'processed': False,
        }
        
    # initialize matrix source values
    matrixLoc[locStart]['distanceFromSource'] = 0
    matrixLoc[locStart]['fromLoc'] = locStart
    matrixLoc[locStart]['processed'] = True
        
    # print(get_itinerary(locStart))
    
    # Find all itinerary from the start location
    # for index, dfloc in get_itinerary(locStart).iterrows():
    #     matrixLoc[dfloc['end']] = {
    #         'distanceFromSource': dfloc['duree'],
    #         'processed': False,
    #     }
    
    updateMatrixNodes(matrixLoc, get_itinerary(locStart))
        
    # trouver le plus petit distanceFromSource qui est processed False pour refaire la meme manip
    
    return matrixLoc
    
    

In [261]:
locStart = "Grenoble"
locEnd = "Le Havre"



dijsktra(itineraries, locStart, locEnd)



                     trip_id  duree             start                      end
57    OCESN017500F0200216492     74  Gare de Grenoble    Gare de Valence-Ville
63    OCESN017600F0300316547     82  Gare de Grenoble   Gare de Lyon-Part-Dieu
1476  OCESN885111F0200250083     40  Gare de Grenoble     Gare de St-Marcellin
1479  OCESN885413F0400450120     62  Gare de Grenoble  Gare de St-André-le-Gaz
1480  OCESN885456F0400450135     30  Gare de Grenoble            Gare de Rives
1483  OCESN885605F0600650145    134  Gare de Grenoble   Gare de Veynes-Dévoluy
infinite
74
infinite
82
infinite
40
infinite
62
infinite
30
infinite
134


{'Gare de Le Havre': {'distanceFromSource': 'infinite',
  'fromLoc': '',
  'processed': False},
 'Gare de Dieppe': {'distanceFromSource': 'infinite',
  'fromLoc': '',
  'processed': False},
 'Gare de Paris-St-Lazare': {'distanceFromSource': 'infinite',
  'fromLoc': '',
  'processed': False},
 'Gare de Cherbourg': {'distanceFromSource': 'infinite',
  'fromLoc': '',
  'processed': False},
 'Gare de Caen': {'distanceFromSource': 'infinite',
  'fromLoc': '',
  'processed': False},
 'Gare de Granville': {'distanceFromSource': 'infinite',
  'fromLoc': '',
  'processed': False},
 'Gare de Dreux': {'distanceFromSource': 'infinite',
  'fromLoc': '',
  'processed': False},
 'Gare de Bourges': {'distanceFromSource': 'infinite',
  'fromLoc': '',
  'processed': False},
 'Gare de Paris-Austerlitz': {'distanceFromSource': 'infinite',
  'fromLoc': '',
  'processed': False},
 'Gare de Nevers': {'distanceFromSource': 'infinite',
  'fromLoc': '',
  'processed': False},
 'Gare de Argentan': {'distanceFrom